## Optimized Model: Rethinking the Pipeline

In the previous notebooks, I implemented multiple models of increasing complexity, ranging from logistic regression to deeper neural networks. While these experiments were valuable for understanding core concepts in machine learning and deep learning, the resulting performance remained relatively low and inconsistent.

After analyzing the results more carefully, it became clear that the limitation was not only related to model architecture or optimization techniques, but also to **how the data was labeled and interpreted**.

### Identified Issue with Data Labeling

In the earlier approach, an image was classified as *damaged* if **any single polygon** within the image was labeled as `D_Building` or `Debris`. This means that even a small, localized damaged region could cause the entire image to be labeled as damaged.  
Such a strategy likely introduces noise and label ambiguity, especially for images that are largely intact but contain minor damage.

This coarse labeling scheme may prevent the model from learning meaningful visual patterns related to *overall structural damage*, which is the core objective of this project.

### Objective of This Notebook

In this notebook, I aim to build the **most optimized model so far**, not only by:
- improving model architecture,
- applying better initialization, regularization, and optimization techniques,

but also by **revisiting and refining the data labeling strategy itself**.

By aligning the labels more closely with the true semantic meaning of structural damage, the goal is to provide the model with cleaner supervision and enable more reliable learning.

This step marks a transition from experimenting with models to **systematically improving the full machine learning pipeline**, from data understanding to final evaluation.
